In [1]:
# Connect to Ollama running Llama3
import dspy
llama3_ollama = dspy.OllamaLocal(model="llama3:instruct", max_tokens=4000, timeout_s=480)

dspy.settings.configure(lm=llama3_ollama)

llama3_ollama("say hello")

/Users/cshorten/Library/Caches/pypoetry/virtualenvs/etl-rZpcbSY7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


["Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"]

In [2]:
# Load blogs into Weaviate
import weaviate

weaviate_client = weaviate.connect_to_local()

weaviate_client.collections.delete("WeaviateBlogChunks")

In [3]:
import weaviate.classes.config as wvcc

collection = weaviate_client.collections.create(
    name="WeaviateBlogChunk",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_cohere
    (
        model="embed-multilingual-v3.0"
    ),
    properties=[
            wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
            wvcc.Property(name="query", data_type=wvcc.DataType.TEXT, skip_vectorization=True),
      ]
)

In [24]:
import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks

# Example usage
main_folder_path = '../../datasets/weaviate-blogs'
blog_chunks = read_and_chunk_index_files(main_folder_path)

print(f"{len(blog_chunks)}\n")
print(blog_chunks[0])

1182

---
title: Combining LangChain and Weaviate
slug: combining-langchain-and-weaviate
authors: [erika]
date: 2023-02-21
tags: ['integrations']
image: ./img/hero.png
description: "LangChain is one of the most exciting new tools in AI. It helps overcome many limitations of LLMs, such as hallucination and limited input lengths."
---
![Combining LangChain and Weaviate](./img/hero.png)

Large Language Models (LLMs) have revolutionized the way we interact and communicate with computers. These machines can understand and generate human-like language on a massive scale. LLMs are a versatile tool that is seen in many applications like chatbots, content creation, and much more. Despite being a powerful tool, LLMs have the drawback of being too general.


In [25]:
from weaviate.util import get_valid_uuid
from uuid import uuid4
import time

blogs = weaviate_client.collections.get("WeaviateBlogChunk")

blog_chunk_uuids = []

start = time.time()
for idx, blog_chunk in enumerate(blog_chunks):
    id = get_valid_uuid(uuid4())
    blog_chunk_uuids.append(id)
    upload = blogs.data.insert(
        properties={
            "content": blog_chunk
        },
        uuid=id
    )

print(f"Uploaded {len(blog_chunks)} blog chunks in {time.time() - start} seconds.")

Uploaded 1182 blog chunks in 173.13162684440613 seconds.


In [124]:
from pydantic import BaseModel
from typing import Any
import functools

class UpdateProperty(dspy.Signature):
    """I need your help to generate the value of a property by following the instruction using the provided name-value property references. VERY IMPORTANT!! Please follow this next instruction carefully. It is EXTREMELY IMPORTANT that you only output the property value and nothing else. Do not start your response with something like `Sure, I can help with that!` or anything of the sort. JUST OUTPUT THE PROPERTY VALUE!!
    """

    property_name: str = dspy.InputField(
        desc="The name of the property that you should update."
    )
    references: str = dspy.InputField(
        desc="The name-value property pairs that you should refer to while updating the property."
    )
    instruction: str = dspy.InputField(
        desc="The prompt to use when generating the updated property value."
    )
    property_value: str = dspy.OutputField(
        desc="The value of the updated property. VERY IMPORTANT!! ONLY OUTPUT THIS VALUE!! Do not output anything other than this value."
    )

class Program(dspy.Module):
    def __init__(self, property_value_type: Any) -> None:
        self.property_value_type = property_value_type
        self.update_property = dspy.Predict(UpdateProperty)

    def assert_property_value_type(self, property_value: str) -> bool:
        try:
            self.property_value_type(property_value)
            return True
        except (ValueError, TypeError):
            return False

    def failed_assertion_message(self, property_name: str) -> str:
        return f"""
        The value of the '{property_name}' property does not match the expected type: {self.property_value_type}.
        Please ensure that the generated value adheres to the specified type.
        """

    def forward(self, property_name: str, references: str, instruction: str) -> Any:
        prediction: dspy.Prediction = self.update_property(
            property_name=property_name, references=references, instruction=instruction
        )

        dspy.Suggest(
            self.assert_property_value_type(prediction.property_value),
            self.failed_assertion_message(property_name),
        )

        return self.property_value_type(prediction.property_value)

In [125]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

program = Program(property_value_type=int)
program_with_assertions = assert_transform_module(
    program, functools.partial(backtrack_handler, max_backtracks=1)
)

property_name = "age"
references = "name: John, occupation: Engineer"
instruction = "Update the 'age' property to a random integer between 25 and 35."

result = program_with_assertions(
    property_name=property_name, references=references, instruction=instruction
)
print(result)

29


In [132]:
program = Program(property_value_type=str)

instruction = """
Write a question that the content contains the answer to. IMPORTANT! Make the question specific to the content!!
"""

### Benchmark GFL Speed

In [134]:
start_gfl = time.time()

for idx, chunk_uuid in enumerate(blog_chunk_uuids):
    if idx % 100 == 99:
        print(f"\nLOG: {idx+1} queries generated in {time.time() - start_gfl} seconds.\n")
    # Get the object
    
    object = blogs.query.fetch_object_by_id(chunk_uuid, return_properties="content")
    
    # Format the references
    
    references=" ".join(f"{k}: {v}" for k, v in object.properties.items())
    
    # Run GFL
    
    query = program(
        property_name="query",
        references=references,
        instruction=instruction,
    )
    
    if idx < 10:
        if idx == 0:
            print("Printing some queries to get a sense of what this is doing... \n")
        print(query)

    # Update property in Weaviate
    
    blogs.data.update(
        properties={
            "query": query
        },
        uuid=chunk_uuid
    )

print(f"{len(blog_chunk_uuids)} objects have been updated in {time.time() - start} seconds.")

Printing some queries to get a sense of what this is doing...
What are some limitations of Large Language Models (LLMs)?
What are some limitations of LLMs and how do emerging technologies help solve them?
What are some key concepts introduced in LangChain?
What type of mammal lays the biggest eggs?
What are LLM Chains?
What is the simplest method for passing related documents from the database into the language model?
What are the main techniques used in the LLM to generate multiple responses?
What is the process involved in Map Rerank?
What is an example of tool use in language models, and how can it be used to augment their capabilities?
What is the purpose of Python REPL in LangChain?

LOG: 100 queries generated in 154.4688901901245 seconds.


LOG: 200 queries generated in 286.906888961792 seconds.


LOG: 300 queries generated in 436.4760699272156 seconds.


LOG: 400 queries generated in 575.7406079769135 seconds.


LOG: 500 queries generated in 708.9861929416656 seconds.


LOG: 600

## What can we do with these synthetic queries?

- Benchmarking Search

- Train Custom Search Models with this Dataset

### Benchmark Search

### WORK IN PROGRESS

In [ ]:
queries_and_ground_truth_ids = []

for chunk in blog_chunk_uuids:
    object = blogs.query.fetch_object_by_id(chunk_uuid, return_properties="content")
    for property in object.properties:
        # query
        # uuid

search_results = []
for obj in queries_and_ground_truth_ids:
    # search
    # where is the id in the search results?

# calculate recall at 1,5,10, ...

### Fine-Tune Cohere Re-Ranker with Synthetic Queries

In [ ]:
# Optimize query generation with PATH algorithm (https://arxiv.org/abs/2406.11706)

# Use queries to train a reranker with Cohere's fine-tuning API